# 계층형 조회(Connect By)

- 계층형 조회는 Oracle 데이터베이스에서 지원하는 것으로 계층형으로 데이터를 조회할 수 있음
- 트리 형태의 구조를 위에서 아래로 탐색하면서 조회하는 것으로, 역방향도 가능함
- Connect by는 트리 형태의 구조로 질의를 수행하는 것으로 START WITH구는 시작 조건을 의미하고 CONNECT BY PRIOR는 조인 조건이다. Root 노드부터 하위 노드의 질의를 실행한다.
- 계층형 조회에서 MAX(LEVEL)을 사용하여 최대 계층 수를 구할 수 있다. 즉, 계층형 구조에서 마지막 Leaf Node의 계층값을 구한다.

## 최대 계층값 계산

```SQL
SELECT MAX(LEVEL)
 FROM Limbest.EMP
START WITH MGR IS NULL
CONNECT BY PRIOR EMPNO = MGR; 
```

## 계층 조회

```SQL
SELECT LEVEL, EMPNO, MGR, ENAME
 FROM Limbest.EMP
START WITH MGR IS NULL
CONNECT BY PRIOR EMPNO = MGR;
```

## 계층 시각화 

- LPAD 함수를 사용해 ROot 계층은 제일 앞에, LEVEL이 증가할수록 키를 뒤에 배치한다. 

```SQL
SELECT LEVEL, LPAD(' ', MAX(LEVEL) * (LEVEL - 1))||EMPNO,
 MGR, CONNECT_BY_ISLEAF
 FROM EMP
START WITH MGR IS NULL
CONNECT BY PRIOR EMPNO = MGR
```

## CONNECT BY 키워드 

- LEVEL : 검색 항목의 깊이를 의미함(상위 레벨이 1)
- CONNECT_BY_ROOT : 계층구조에서 가장 최상위 값(루트)을 표시
- CONNECT_BY_ISLEAF : 계층구조에서 가장 최하위 값(리프)을 표시
- SYS_CONNECT_BY_PATH : 계층구조의 전체 전개 경로를 표시
- NOCYCLE : 순환구조가 발생지점까지만 전개
- CONNECT_BY_ISCYCLE : 순환구조 발생 지점을 표시
- ORDER SIBLINGS BY : 전체 테이블 기준이 아닌 계층별로 정렬함 
- NO CYCLE : 중복이 발생했을 때 루프를 돌지 않기 위해 사용 

# 서브쿼리(Subquery) 

## Main query와 Subquery

- SELECT문 내에 다시 SELECT문을 사용하는 SQL문
- FROM구에 SELECT문을 사용하는 Inline View 와 SELECT문에 Subquery를 사용하는 Scala Subquery가 존재
- WHERE구에 SELECT문을 사용하면 Nested Subquery

```SQL
SELECT *
 FROM EMP
WHERE DEPTNO = (SELECT DEPTNO FROM DEPT WHERE DEPTNO=10);

SELECT *
 FROM (SELECT ROWNUM NUM, ENAME FROM EMP) AS A
WHERE NUM < 5;
```

## 단일 행 서브쿼리와 다중 행 서브쿼리 

- 서브쿼리는 반환하는 행 수가 한개인 것과 여러 개인 것에 따라서 단일 행 서브쿼리와 멀티 행 서브쿼리로 분류
- 단일 행 서브쿼리는 단 하나의 행만 반환하는 서브쿼리로 비교 연산자를 사용함
- 다중 행 서브쿼리는 여러 개의 행을 반환하는 것으로 IN, ANY, ALL, EXISTS를 사용해야 함

## 다중 행 서브쿼리 

**1. IN**
- IN은 반환되는 여러 개의 행 중에서 하나만 참이 되는 연산

```SQL
SELECT ENAME, DNAME, SAL
 FROM EMP, DEPT
WHERE EMP.DEPTNO = DEPT.DEPTNO
  AND EMP.ENAME IN (SELECT EMPNO FROM EMP WHERE SAL > 2000);
```

**2. ALL**
- ALL은 메인쿼리와 서브쿼리의 결과가 모두 동일하면 참이 됨
- 아래는 최솟값 30보다 작거나 같은 모든 행 출력

```SQL
SELECT DISTINCT DEPARTMENT_ID
  FROM HR.EMPLOYEES A
 WHERE A.DEPARTMENT_ID <= ALL (30, 50);
```

**3. EXISTS**
- 어떤 데이터 존재 여부를 확인하는 서브쿼리
- 참과 거짓을 반환

```SQL
SELECT ENAME, DNAME, SAL
  FROM EMP, DEPT
 WHERE EMP.DEPTNO = DEPT.DEPTNO
   AND EXISTS (SELECT 1 FROM EMP WHERE SAL > 2000);
```

## Scala Subquery
- Scala Subquery는 반드시 한 행과 한 컬럼만 반환하는 서브쿼리 

```SQL
SELECT ENAME, SAL, (SELECT AVG(SAL) FROM EMP) AS AVG_SAL
 FROM EMP
WHERE EMPNO=1000;
```

## Correlated Subquery
- Subquery 내에서 Main query 내의 컬럼을 사용하는 서브쿼리 

```SQL
SELECT *
  FROM EMP AS A
 WHERE EMP.DEPTNO = (SELECT DEPTNO FROM DEPT AS B
 WHERE B.DEPTNO = A.DEPTNO);
```

# Group Function

GROUP BY에 입력된 컬럼들을 기준으로 한쪽만 그룹핑 한 합계, 양쪽을 그룹핑한 합계를 모두 고려하여 테이블에 표시하였다. 전체 합계의 경우에만 양쪽의 값이 NULL로 표기되었다.

## ROLLUP

- ROLLUP은 GROUP BY의 컬럼에 대해서 Subtotal(전체 합)을 만들어준다.
- ROLLUP을 할 때 GROUP BY구에 칼럼이 두 개 이상 오면 순서에 따라서 결과가 달라진다.
- 새로운 행을 생성하는 것이므로 DECODE를 사용하여 NULL값의 행을 채워줄 수 있다.
- 첫번째 컬럼을 기준으로 집계함

```SQL
SELECT DECODE(DEPTNO, NULL, '전체합계', DEPTNO), SUM(SAL)
  FROM EMP
 GROUP BY ROLLUP (DEPTNO);
 
SELECT DEPTNO, JOB, SUM(SAL)
  FROM EMP
 GROUP BY ROLLUP (DEPTNO, JOB);
```

## GROUPING 

- GROUPING 함수는 ROLLUP, CUBE, GROUPING SET에서 생성되는 합계값을 구분하기 위해서 만들어진 함수
- 소계, 합계 등이 계산되면 GROUPING 함수는 1을 반환하고 그렇지 않으면 0을 반환하여 합계값을 구분 가능
- 첫번째 컬럼을 기준으로 집계함

```SQL
SELECT DEPTNO, GROUPING(DEPTNO), JOB, GROUPING(JOB), SUM(SAL)
  FROM EMP
 GROUP BY ROLLUP (DEPTNO, JOB);
 
SELECT DEPTNO, DECODE(GROUPING(DEPTNO), 1, '전체합계') TOT, JOB,
 DECODE(GROUPING(JOB), 1, '부서합계') T_DEPT, SUM(SAL)
  FROM EMP
 GROUP BY ROLLUP (DEPTNO, JOB);
```

## GROUPING SETS

- GROUPING SETS 함수는 GROUP BY에 나오는 컬럼의 순서와 관계없이 다양한 소계를 만들 수 있음
- GROUP BY에 나오는 컬럼의 순서와 관계없이 개별적으로 모두 처리
- 각각의 컬럼으로 GROUP BY를 진행한 테이블을 행을 기준으로 합친 것과 동일한 결과를 출력
- 전체 합계가 조회되지 않을 때 사용 GROUPING SETS(COL1, (COl1, COL2))

```SQL
SELECT DEPTNO, JOB, SUM(SAL)
  FROM EMP
 GROUP BY GROUPING SETS (DEPTNO, JOB);
```

## CUBE

- CUBE 함수에 제시한 컬럼에 대해서 결합 가능한 모든 집계를 계산함
- 다차원 집계를 제공하여 다양하게 데이터를 분석할 수 있음
- 조합할 수 있는 경우의 수가 모두 조합되는 것

```SQL
SELECT DEPTNO, JOB, SUM(SAL)
  FROM EMP
 GROUP BY CUBE(DEPTNO, JOB);
```

# Window Function

## What is Window Function?

**1. 구조**
- 윈도우 함수는 행과 행 간의 관계를 정의하기 위해서 제공되는 함수
- 순위, 합계, 평균, 행 위치를 조작 가능

```SQL
SELECT WINDOW_FUNCTION(ARGUMENTS)
  OVER (PARTITION BY [col] ORDER BY [windowing])
  FROM [table];
```

**2. 함수구조**
- ARGUMENTS : 윈도우 함수에 따라 0~N개의 인수를 설정
- PARTITION BY : 전체 집합을 기준에 의해 소그룹으로 분할
- ORDER BY : 기준에 대하여 정렬
- WINDOWING : 행 기준의 범위를 설정

**3. WINDOWING**
- ROWS : 부분집합인 윈도우 크기를 물리적 단위로 행 집합을 지정
- RANGE : 논리적인 주소에 의해 행 집합을 지정
- BETWEEN ~ AND : 윈도우의 시작과 끝의 위치를 지정
- UNBOUNDED PRECEDING : 윈도우의 시작 위치가 첫번째 행
- UNBOUNDED FOLLOWING : 윈도우의 마지막 위치가 마지맥 행
- CURRENT ROW : 윈도우의 시작위치가 현재 행

```SQL
/* 처음 행부터 마지막행까지의 합을 계산*/
SELECT EMPNO, ENAME, SAL,
   SUM(SAL) OVER (ORDER BY SAL 
                  ROWS BETWEEN UNBOUNDED PRECEDING AND UNBOUNDED FOLLOWING) AS TOTSAL
  FROM EMP;

/* 행 별 누적 합계 계산*/
SELECT EMPNO, ENAME, SAL
 SUM(SAL) OVER (ORDER BY SAL
                ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) AS TOTSAL
  FROM EMP
```

## Rank Function
- 특정항목과 파티션에 대해서 순위를 계산할 수 있는 함수를 제공
- RANK : 순위를 계산하여 값을 부여함(즉 2위가 2명이면 다음 순위는 4위가 됨)
- DENSE_RANK : 동일한 순위를 하나의 건수로 계산함(즉 2위가 2명이면 다음 순위는 3위가 됨)
- ROW_NUMBER : 동일한 순위에 대해서 고유한 순위를 부여함

```SQL
/*ALL_RANK : 전체 급여 순위를 계산함*/
/*JOB_RANK : 직업별 순위를 계산함*/
SELECT ENAME, SAL
 RANK() OVER (ORDER BY SAL DESC) ALL_RANK,
 RANK() OVER (PARTITION BY JOB ORDER BY SAL DESC) JOB_RANK
  FROM EMP;

SELECT ENAME, SAL,
 DENSE_RANK() OVER (ORDER BY SAL DESC) DENSE_RANK
  FROM EMP;

SELECT ENMAE, SAL,
 ROW_NUMBER() OVER(ORDER BY SAL DESC) ROW_NUM;
 FROM EMP;
```

## 행 순서 관련 함수 
- 상위 행의 값을 하위에 출력하거나 하위 행의 값을 상위 행에 출력할 수 있음
- FIRST_VALUE : 파티션에서 가장 처음에 나오는 값을 출력
- LAST_VALLUE : 파티션에서 가장 나중에 나오는 값을 출력
- LAG : 이전 행을 가지고 옴
- LEAD(행, 순서, 디폴트) : 특정 위치의 행을 가지고 옴

```SQL
/*DEPTNO에 파티션된 행 중에서 첫번째 행의 값을 가지고 온다*/
SELECT DEPTNO, ENAME, SAL,
 FIRST_VALUE(ENAME) OVER (PARTITION BY DEPTNOORDER BY SAL DESC 
                          ROWS UNBOUNDED_PRECEDING) AS DEPT_A
 FROM EMP;
 
SELECT DEPTNO, ENAME, SAL
 LAST_VALUE(ENAME) OVER (PARTITION BY DEPTNO ORDER BY SAL DESC
                         ROWS BETWEEN CURRENT ROW AND UNBOUNDED FOLLLOWING) AS DEPT_A
 FROM EMP;
```

## 비율 관련 함수 
- CUME_DIST : 파티션 전체 건수에서 현재 행보다 작거나 같은 건수에 대한 누적 백분율을 조회
- PERCENT_RANK : 파티션에서 제일 먼저 나온 것을 0으로, 제일 늦게 나온것을 1로하여 갑이 아닌 행의 순서별 백분율을 조회
- NTITLE : 파티션별로 전체 건수를 ARGUMENT 값으로 N 등분한 결과를 조회
- RATIO_TO_REPORT : 파티션 내에 전체 컬럼에 대한 행 별 컬럼값의 백분율을 소수점까지 조회

```SQL
/* 같은 부서에서 자신의 급여 퍼센트(등수)를 계산*/
SELECT DEPTNO, ENAME, SAL,
 PERECENT_RANK() OVER (PARTITION BY DEPTNO ORDER BY SAL DESC) AS PERCENT_SAL
 FROM SAL;

/* 급여가 높은순으로 4단계로 분류*/
SELECT DEPTNO, ENAME, SAL, NTITLE(4) OVER (ORDER BY SAL DESC) AS N_TITLE
 FROM EMP;
```

# 테이블 파티션(Table Partition) 

## Partition 기능 
- 대용량의 테이블을 여러 개의 데이터 파일에 분리해서 저장
- 테이블의 데이터가 물리적으로 분리된 데이터 파일에 저장되면 입력, 수정, 삭제, 조회 성능이 향상됨
- 각각의 파티션 별로 독립적으로 관리가능함
- 데이터를 조회할 때 데이터의 범위를 줄여서 성능을 향상시킴(Pandas의 Chunk와 비슷)

## Partition Range
- 테이블의 컬럼 중에서 값의 범위를 기준으로 여러 개의 파티션으로 데이터를 저장함
- 대상 테이블이 날짜/숫자값으로 분리가 가능하고 각 영역별로 트랜잭션이 분리되면 적용 가능
- 데이터 보관 주기에 따라 테이블에 데이터를 쉽게 지우는 것이 가능하므로 데이터 보관 주기에 따른 테이블 관리가 용이함

## List Partition
- 특정 값을 기준으로 분할하는 방법
- 핵심적인 코드값 등으로 기본키가 구성되어 있고 대량의 데이터가 있는 테이블에서 이용 가능
- 데이터 보관 주기에따라 쉽게 삭제하는 기능을 제공하지는 않음

## Hash Partition 
- 데이터베이스 관리 시스템이 내부적으로 해시 함수를 사용해서 데이터를 분할
- 데이터베이스 관리 시스템이 알아서 분할하고 관리하는 파티션
- 설계자는 테이블에 데이터가 정확하게 어떻게 들어가는지 알 수 없음

## Partition Index 
- Global Index : 여러 개의 파티션에서 하나의 인덱스를 사용
- Local Index : 해당 파티션 별로 각자의 인덱스를 사용
- Prefixed Index : 파티션 키와 인덱스 키가 동일
- Non Prefixed Index : 파티션 키와 인덱스 키가 서로 다름